In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [2]:
sms = pd.read_csv("SMSSpamCollection.txt", sep = '\t', header = None)

In [3]:
sms.columns = ['tag', 'text']

In [4]:
sms.head()

,tag,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [5]:
text = sms.text.values
y = [1 if x == 'spam' else 0 for x in sms.tag.values ]

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [15]:
lr_lib = LogisticRegression(random_state = 2, solver = 'liblinear')
score = cross_val_score(lr_lib, X, y, cv = 10, scoring = 'f1')

In [9]:
score.mean()

0.9326532471300417

In [10]:
test = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]

In [11]:
lr = LogisticRegression(random_state = 2)
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
lr.predict(vectorizer.transform(test))

array([1, 1, 0, 0, 0])

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [16]:
v_1 = CountVectorizer(ngram_range=(2,2))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(lr_lib, y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.8216825243287891

In [29]:
v_1 = CountVectorizer(ngram_range=(3,3))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(lr_lib, y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.7249691484935827

In [18]:
v_1 = CountVectorizer(ngram_range=(1, 3))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(lr_lib, y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.9251702399813329

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [23]:
v_1 = CountVectorizer(ngram_range=(2,2))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(MultinomialNB(), y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.6457761003991855

In [24]:
v_1 = CountVectorizer(ngram_range=(3,3))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(MultinomialNB(), y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.37861105465217015

In [25]:
v_1 = CountVectorizer(ngram_range=(1,3))
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(MultinomialNB(), y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.888454144419353

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [26]:
v_1 = TfidfVectorizer()
y_1 = v_1.fit_transform(text)
score_1 = cross_val_score(lr_lib, y_1, y, scoring = 'f1', cv = 10)
score_1.mean()

0.8528288475348873